In [12]:
pip install selenium beautifulsoup4 requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\wjnar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import re
import requests

In [14]:
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run headless 
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # Prevent bot detection
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service=Service(), options=chrome_options)

url = "https://www.redfin.com/"
driver.get(url)

search_term = "831 Crescent Dr, Boulder, CO 80303"

# search_input = driver.find_element(By.ID, "search-box-input")
search_input = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "search-box-input"))
)

search_input.send_keys(search_term)

search_input.send_keys(Keys.RETURN)

In [15]:
property_details = {}

WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "AgentInfoCard"))
)


agent_name = driver.find_element(By.CLASS_NAME, "agent-basic-details--heading").text.strip()
property_details["Agent Name"] = agent_name

brokerage_name = driver.find_element(By.CLASS_NAME, "agent-basic-details--broker").text
property_details["Brokerage Name"] = brokerage_name


In [16]:
contact_section = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "listingContactSection"))
)

# Contact info can range from email, phone, both, etc. its going to be hard to standardize
# Maybe use regex to use it in the future
contact_info = contact_section.text.replace("Contact: ", "").strip()
property_details["Agent Contact Info"] = contact_info

In [17]:
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "ExpandableAmenitiesInfoRow"))
)
sections = driver.find_elements(By.CLASS_NAME, "ExpandableAmenitiesInfoRow")

soup = BeautifulSoup(driver.page_source, "html.parser")

amenity_groups = soup.find_all("div", class_="amenity-group") 
for group in amenity_groups:
    
    section_title_element = group.find("div", class_="propertyDetailsHeader")
    section_title = section_title_element.get_text(strip=True) if section_title_element else "Unknown"

    details = group.find_all("li", class_="entryItem")
    # print(f"Found {len(details)} details in section '{section_title}'")

    for detail in details:
        label_element = detail.find("span", class_="entryItemContent")

        if label_element:
            # print(f"Label Element Found: {label_element.text}")  # Debugging
            label = label_element.text.split(":")[0]
            
            spans = label_element.find_all("span")
            # print(f"Found {len(spans)} spans in label")  # Debugging

            if len(spans) > 0:
                detail_value = spans[-1].get_text(strip=True)  # Last span is the value
                property_details[label] = detail_value
            else:
                print(f"Skipping detail (Not enough spans): {label_element.text}")

        else:
            print("Skipping detail (Label element not found)")

In [18]:
for key, value in property_details.items():
        if key != value:
                print(f"{key}: {value}")

Agent Name: Lex Campaldini
Brokerage Name: Redfin Corporation
Agent Contact Info: Lex.Campaldini@redfin.com,720-448-8539
Parking Total: 5
Off Street Spaces: 2
Garage Spaces: 3
AttachedGarageYN: Yes
Parking Features: Concrete, Exterior Access Door, Heated Garage, Insulated Garage, Oversized, Storage
Bedrooms Total: 3
Upper Level Bedrooms: 1
Main Level Bedrooms: 2
Bathrooms Total Integer: 3
Bathrooms Full: 2
Bathrooms Half: 1
Upper Level Bathrooms: 1
Main Level Bathrooms: 2
Level: Upper
Description: Tile flooring, five-piece ensuite
Interior Features: Built-in Features,Five Piece Bath,Laminate Counters,Open Floorplan,Pantry,Primary Suite,Quartz Counters,Smoke Free,Utility Sink,Vaulted Ceiling(s),Walk-In Closet(s)
Appliances: Cooktop, Dishwasher, Disposal, Dryer, Microwave, Oven, Refrigerator, Sump Pump, Washer
Flooring: Carpet, Tile, Wood
Furnished: Unfurnished
Basement: Crawl Space
Fireplaces Total: 1
Fireplace Features: Living Room
Building Area Total: 2,597
Building Area Source: Publi

: 